In [1]:
import import_ipynb
import socket
import threading
import time
from tkinter import *
# import tkinter
from tkinter import messagebox
import pickle
import mse_review
import binascii

name=input("Enter your name : ")
public, private = mse_review.generate_large_no(1024)
msg=pickle.dumps(public)
#print(public[0])

def set_ip():
    ip = edit_text_ip.get()
    port = edit_text_port.get()
    
    # Define Server:
    server = socket.socket()
    server.bind((ip, int(port)))
    server.listen()

    global conn
    conn, addr = server.accept()

    input_root.destroy()     # destroy window
    input_root.quit()           # end window


def send():
    if str(edit_text.get()).strip() != "":
        message = str.encode(edit_text.get())
        #converting it into numb
        hex_data   = binascii.hexlify(message)
        plain_text = int(hex_data, 16)
        ctt=mse_review.encrypt(plain_text,pkey)
        if(ctt==1):
            messagebox.askretrycancel("Msg not delivered","Size of message is large")
            send()
        conn.send(str(ctt).encode())

        listbox.insert(END, name + " : " +str(edit_text.get()))               # scrollbar
        edit_text.delete(0, END)
    else:
        messagebox.askretrycancel("Empty Message", "Please type a message..") 


def recv():
    while True:
        response_message =int(conn.recv(1024).decode())
#         print(response_message)
        decrypted_msg = mse_review.decrypt(response_message, private)

        listbox.insert(END, name1 +" : "+ str(decrypted_msg))
        edit_text.delete(0, END)

def close():
    root.destroy()
    root.quit()
        
# first window for setting ip and port no
input_root = Tk()
Label(input_root,bg="#2F4F4F").place(relwidth=1,relheight=1)

edit_text_ip = Entry()
edit_text_port = Entry()
ip_label = Label(input_root, bg="grey",fg="white", text="Host IP:")
port_label = Label(input_root, bg="grey", fg="white", text="Host Port:")
connect_btn = Button(input_root, text="Connect To Server", command=set_ip, bg='#668cff', fg="white")

ip_label.pack(fill=X,pady=10,padx=50, side=TOP)
edit_text_ip.pack(side=TOP)
port_label.pack(fill=X, pady=10,padx=50, side=TOP)
edit_text_port.pack( side=TOP)
connect_btn.pack(fill=X,pady=10,padx=10, side=BOTTOM)

input_root.title(name)
input_root.geometry("300x200")
input_root.resizable(width=True, height=True)

input_root.mainloop()

conn.send(str.encode(name))        # send name of user
name1=conn.recv(1024).decode()
conn.send(msg)                            # send public key
rmsg=conn.recv(1024)                  #receive public key
pkey=pickle.loads(rmsg)
#print("public key of other is :",pkey[0])

# second window after connection established

root = Tk()
scrollbar_h = Scrollbar(root, orient="horizontal")
scrollbar_v = Scrollbar(root)
scrollbar_h.pack(side=BOTTOM,fill=X)
scrollbar_v.pack(side=RIGHT, fill=Y)
listbox = Listbox(root, width=50, height=18, xscrollcommand=scrollbar_h.set, yscrollcommand=scrollbar_v.set, bg='blue')
listbox.pack(fill=BOTH, side=TOP)
scrollbar_h.config(command=listbox.xview)
scrollbar_v.config(command=listbox.yview)
button1 = Button(root, text="Quit", command=close, bg='red', fg="white")
button = Button(root, text="Send Message", command=send, bg='#29a329', fg="white")

button1.pack(fill=X,pady=3,  side=BOTTOM)
button.pack(fill=X, side=BOTTOM)
edit_text = Entry(root)
edit_text.pack(fill=BOTH,side=TOP, expand=TRUE)
scrollbar_h.pack(after=button)

root.title(name)
root.geometry("400x400")
# root.resizable(width=True, height=True)

threading.Thread(target=recv).start()           #simultaneous working of both hosts

root.mainloop()


importing Jupyter notebook from mse_review.ipynb
Enter your name : isha


In [ ]:
# 127.0.0.1 ->(localhost) loopback address
# 0.0.0.0 ->non-routable meta-address